In [160]:
import requests
import sys
import pandas as pd
import json
from pandas.io.json import json_normalize

In [161]:
# openweathermap.org url to pull data from their weather api
url = 'https://api.openweathermap.org/data/2.5/forecast?zip=55016,us&appid=d7fea725e801106f403a9f54ebbf5bff&units=imperial'

# pull api json data and save it as dictionary
data = requests.get(url).json()
data

{u'city': {u'coord': {u'lat': 44.8308, u'lon': -92.9393},
  u'country': u'US',
  u'name': u'Cottage Grove',
  u'sunrise': 1573909811,
  u'sunset': 1573944185,
  u'timezone': -21600},
 u'cnt': 40,
 u'cod': u'200',
 u'list': [{u'clouds': {u'all': 100},
   u'dt': 1573916400,
   u'dt_txt': u'2019-11-16 15:00:00',
   u'main': {u'grnd_level': 988,
    u'humidity': 88,
    u'pressure': 1023,
    u'sea_level': 1023,
    u'temp': 35.31,
    u'temp_kf': -0.13,
    u'temp_max': 35.55,
    u'temp_min': 35.31},
   u'sys': {u'pod': u'd'},
   u'weather': [{u'description': u'overcast clouds',
     u'icon': u'04d',
     u'id': 804,
     u'main': u'Clouds'}],
   u'wind': {u'deg': 163, u'speed': 15.28}},
  {u'clouds': {u'all': 100},
   u'dt': 1573927200,
   u'dt_txt': u'2019-11-16 18:00:00',
   u'main': {u'grnd_level': 987,
    u'humidity': 71,
    u'pressure': 1021,
    u'sea_level': 1021,
    u'temp': 41.13,
    u'temp_kf': -0.1,
    u'temp_max': 41.31,
    u'temp_min': 41.13},
   u'sys': {u'pod': u'd'

In [162]:
# begin parsing dictionary
data.keys()

[u'city', u'message', u'list', u'cod', u'cnt']

In [163]:
data['list'][0].keys()

[u'clouds', u'sys', u'dt_txt', u'weather', u'dt', u'main', u'wind']

In [164]:
# find that the list key brings us to the data that I am looking for
data['list'][0]['main']

{u'grnd_level': 988,
 u'humidity': 88,
 u'pressure': 1023,
 u'sea_level': 1023,
 u'temp': 35.31,
 u'temp_kf': -0.13,
 u'temp_max': 35.55,
 u'temp_min': 35.31}

In [165]:
# create a pandas dataframe from the dictionary
df = pd.DataFrame(json_normalize(data['list']))

In [166]:
df.head()

,clouds.all,dt,dt_txt,main.grnd_level,main.humidity,main.pressure,main.sea_level,main.temp,main.temp_kf,main.temp_max,main.temp_min,rain.3h,snow.3h,sys.pod,weather,wind.deg,wind.speed
0,100,1573916400,2019-11-16 15:00:00,988,88,1023,1023,35.31,-0.13,35.55,35.31,NaN,NaN,d,"[{u'description': u'overcast clouds', u'main':...",163,15.28
1,100,1573927200,2019-11-16 18:00:00,987,71,1021,1021,41.13,-0.10,41.31,41.13,NaN,NaN,d,"[{u'description': u'overcast clouds', u'main':...",170,15.75
2,85,1573938000,2019-11-16 21:00:00,983,68,1018,1018,41.52,-0.07,41.63,41.52,NaN,NaN,d,"[{u'description': u'overcast clouds', u'main':...",170,13.27
3,89,1573948800,2019-11-17 00:00:00,984,83,1018,1018,36.90,-0.04,36.95,36.90,NaN,NaN,n,"[{u'description': u'overcast clouds', u'main':...",166,10.94
4,100,1573959600,2019-11-17 03:00:00,982,88,1017,1017,36.63,0.00,36.63,36.63,NaN,NaN,n,"[{u'description': u'overcast clouds', u'main':...",176,11.34


In [167]:
# the weather column contains a list with a dictionary in it
# I will be seperating it to create its own dataframe and then bringing it back into the main dataframe

# name columns
cols = df['weather'][0][0].keys()
df_weather = pd.DataFrame(columns=cols)

# create dataframe row from each dictionarry list
for row in range(len(df)):
    df_weather = df_weather.append(df['weather'][row][0], ignore_index=True)


In [168]:
#rename columns and remove id columns
df_weather.rename(columns = {'description':'weather.desc', 'main':'weather.main'}, inplace = True)
df_weather = df_weather[['weather.desc', 'weather.main']]
df_weather.head()

,weather.desc,weather.main
0,overcast clouds,Clouds
1,overcast clouds,Clouds
2,overcast clouds,Clouds
3,overcast clouds,Clouds
4,overcast clouds,Clouds


In [169]:
# remove old weather column and add new weather columns
df = pd.concat([df.drop('weather',1),df_weather], axis=1)

In [170]:
df.head()

,clouds.all,dt,dt_txt,main.grnd_level,main.humidity,main.pressure,main.sea_level,main.temp,main.temp_kf,main.temp_max,main.temp_min,rain.3h,snow.3h,sys.pod,wind.deg,wind.speed,weather.desc,weather.main
0,100,1573916400,2019-11-16 15:00:00,988,88,1023,1023,35.31,-0.13,35.55,35.31,NaN,NaN,d,163,15.28,overcast clouds,Clouds
1,100,1573927200,2019-11-16 18:00:00,987,71,1021,1021,41.13,-0.10,41.31,41.13,NaN,NaN,d,170,15.75,overcast clouds,Clouds
2,85,1573938000,2019-11-16 21:00:00,983,68,1018,1018,41.52,-0.07,41.63,41.52,NaN,NaN,d,170,13.27,overcast clouds,Clouds
3,89,1573948800,2019-11-17 00:00:00,984,83,1018,1018,36.90,-0.04,36.95,36.90,NaN,NaN,n,166,10.94,overcast clouds,Clouds
4,100,1573959600,2019-11-17 03:00:00,982,88,1017,1017,36.63,0.00,36.63,36.63,NaN,NaN,n,176,11.34,overcast clouds,Clouds


In [171]:
# identifying that the dataframe shows NaN when there is no rain
df['rain.3h'].isna().sum()

35

In [172]:
# changing the rain column to show 0 when there is no rain rather than NaN
df['rain.3h'].fillna(0, inplace=True)

In [173]:
df.head()

,clouds.all,dt,dt_txt,main.grnd_level,main.humidity,main.pressure,main.sea_level,main.temp,main.temp_kf,main.temp_max,main.temp_min,rain.3h,snow.3h,sys.pod,wind.deg,wind.speed,weather.desc,weather.main
0,100,1573916400,2019-11-16 15:00:00,988,88,1023,1023,35.31,-0.13,35.55,35.31,0.0,NaN,d,163,15.28,overcast clouds,Clouds
1,100,1573927200,2019-11-16 18:00:00,987,71,1021,1021,41.13,-0.10,41.31,41.13,0.0,NaN,d,170,15.75,overcast clouds,Clouds
2,85,1573938000,2019-11-16 21:00:00,983,68,1018,1018,41.52,-0.07,41.63,41.52,0.0,NaN,d,170,13.27,overcast clouds,Clouds
3,89,1573948800,2019-11-17 00:00:00,984,83,1018,1018,36.90,-0.04,36.95,36.90,0.0,NaN,n,166,10.94,overcast clouds,Clouds
4,100,1573959600,2019-11-17 03:00:00,982,88,1017,1017,36.63,0.00,36.63,36.63,0.0,NaN,n,176,11.34,overcast clouds,Clouds


In [174]:
# created a datetime column from unix time in dt
from datetime import datetime
date_time = []
for dt in df['dt']:
    date_time.append(datetime.fromtimestamp(dt))

df['dt'] = date_time

In [175]:
df.head()

,clouds.all,dt,dt_txt,main.grnd_level,main.humidity,main.pressure,main.sea_level,main.temp,main.temp_kf,main.temp_max,main.temp_min,rain.3h,snow.3h,sys.pod,wind.deg,wind.speed,weather.desc,weather.main
0,100,2019-11-16 09:00:00,2019-11-16 15:00:00,988,88,1023,1023,35.31,-0.13,35.55,35.31,0.0,NaN,d,163,15.28,overcast clouds,Clouds
1,100,2019-11-16 12:00:00,2019-11-16 18:00:00,987,71,1021,1021,41.13,-0.10,41.31,41.13,0.0,NaN,d,170,15.75,overcast clouds,Clouds
2,85,2019-11-16 15:00:00,2019-11-16 21:00:00,983,68,1018,1018,41.52,-0.07,41.63,41.52,0.0,NaN,d,170,13.27,overcast clouds,Clouds
3,89,2019-11-16 18:00:00,2019-11-17 00:00:00,984,83,1018,1018,36.90,-0.04,36.95,36.90,0.0,NaN,n,166,10.94,overcast clouds,Clouds
4,100,2019-11-16 21:00:00,2019-11-17 03:00:00,982,88,1017,1017,36.63,0.00,36.63,36.63,0.0,NaN,n,176,11.34,overcast clouds,Clouds
